In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data
import torchvision
from torchvision import transforms

In [2]:
import sys

COLAB = 'google.colab' in str(get_ipython())

if COLAB: # running on Colab
    
    from google.colab import drive
    drive.mount('/content/drive')
    gdrive_path = '/content/drive//MyDrive/DL4CV-2022/project-I/'
    sys.path.append(gdrive_path)

else:
    path = 'data'
    
import utils.helpers as utils
import loader.gtsrb_data as dataset
    

In [3]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--data_dir_train', type=str, default='data/Final_Training/Images',
                   help='data directory containing training image class folders')
parser.add_argument('--data_dir_test', type=str, default='data/Final_Test',
                   help='data directory containing test images')
parser.add_argument('--annotations_file', type=str, default='data/Final_Training/Annotations',
                   help='data directory containing class annotations file')
parser.add_argument('--data_dir_val', type=str, default='data/Final_Validation',
                   help='data directory containing validation images')
parser.add_argument('--log_dir', type=str, default='logs',
                   help='directory containing logs')
parser.add_argument('--save_dir', type=str, default='save',
                   help='directory to store checkpointed models')
parser.add_argument('--batch_size', type=int, default=50,
                   help='minibatch size')
parser.add_argument('--num_epochs', type=int, default=5,
                   help='number of epochs')
parser.add_argument('--save_every', type=int, default=1000,
                   help='save frequency')
parser.add_argument('--learning_rate', type=float, default=0.002,
                   help='learning rate')
parser.add_argument('--decay_rate', type=float, default=0.97,
                   help='decay rate for rmsprop')
parser.add_argument('--gpu_mem', type=float, default=0.666,
                   help='%% of gpu memory to be allocated to this process. Default is 66.6%%')
parser.add_argument('--init_from', type=str, default=None,
                   help="""continue training from saved model at this path. Path must contain files saved by previous training process:
                        'config.pkl'        : configuration;
                        'words_vocab.pkl'   : vocabulary definitions;
                        'checkpoint'        : paths to model file(s) (created by tf).
                                              Note: this file contains absolute paths, be careful when moving files around;
                        'model.ckpt-*'      : file(s) with model definition (created by tf)
                    """)
args = parser.parse_args(args=[])
print(args)

Namespace(annotations_file='data/Final_Training/Annotations', batch_size=50, data_dir_test='data/Final_Test', data_dir_train='data/Final_Training/Images', data_dir_val='data/Final_Validation', decay_rate=0.97, gpu_mem=0.666, init_from=None, learning_rate=0.002, log_dir='logs', num_epochs=5, save_dir='save', save_every=1000)


In [4]:
#transform=TRANSFORM_IMG
transform=None

if COLAB:
    TRAIN_DATA_PATH = gdrive_path+args.data_dir_train
    VAL_DATA_PATH = gdrive_path+args.data_dir_val
    TEST_DATA_PATH = gdrive_path+args.data_dir_test
    print(gdrive_path+args.data_dir_test)
    print(gdrive_path+args.annotations_file)

else:
    TRAIN_DATA_PATH = os.getcwd()+'/'+args.data_dir_train
    VAL_DATA_PATH = os.getcwd()+'/'+args.data_dir_val
    TEST_DATA_PATH = os.getcwd()+'/'+args.data_dir_test
    
BATCH_SIZE = args.batch_size
EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
LEARNING_RATE = args.learning_rate


In [5]:
train_data        = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=None)
train_data_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2)
test_data         = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=None)
test_data_loader  = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_data          = torchvision.datasets.ImageFolder(root=VAL_DATA_PATH, transform=None)
val_data_loader   = data.DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2)

In [6]:
print(len(train_data))
print(len(test_data))
print(len(val_data))

39209
12630
3870


In [7]:
train_class_dict = train_data.class_to_idx
val_class_dict = val_data.class_to_idx
test_class_dict = test_data.class_to_idx # A 'single' class